# Utility to create stable file used for reproduciblity checks

## v2.1 Code

In [ ]:
import pickle
import numpy as np
import os.path as osp
import os
import pandas as pd
import tensorflow as tf
import sys
sys.path.append('..')
from moisture_rnn_pkl import pkl2train
from moisture_rnn import RNNParams
from data_funcs import build_train_dict
from utils import read_yml, read_pkl, print_dict_summary, retrieve_url

In [ ]:
filename = "test_CA_202401.pkl"
retrieve_url(
    url = f"https://demo.openwfm.org/web/data/fmda/dicts/{filename}", 
    dest_path = f"../data/{filename}")

In [ ]:
pkl_file = f"../data/{filename}"
case_name = "NV020_202401"
# Destination File
outfile = "../data/reproducibility_dict_v2_TEST.pkl"

## Read Data and Extract Case

In [ ]:
train = read_pkl(pkl_file)

In [ ]:
params_data = read_yml("../params_data.yaml")

In [ ]:
params_data.update({'hours': 168})

In [ ]:
train = build_train_dict([pkl_file], params_data=params_data,
                         spatial=False)

In [ ]:
print_dict_summary(train[case_name])

## Add Reproducibility Info

In [ ]:
params = read_yml('../params.yaml', subkey="rnn_repro")
params

In [ ]:
repro_info = {
    'phys_initialize': "NOT YET IMPLEMENTED WITH v2.1",
    'rand_initialize':{
        'fitted_weights_hash': '01513ac086d842dc67d40eb94ee1110c',
        'preds_hash': '4999d10893207f2b40086e3f84c214a3'
    },
    'env_info':{
        'py_version': sys.version[0:6],
        'tf_version': tf.__version__,
        'seed': 123
    },
    'params': RNNParams(params)
}

train[case_name]['repro_info'] = repro_info

## Write Output

In [ ]:
with open(outfile, 'wb') as file:
    print(f"Writing file: {outfile}")
    pickle.dump(train[case_name], file)